## 7.32 异常和错误处理

运行时错误会导致异常，这些异常以陷阱或软件中断的形式被检测到。
代码中使用`try-catch`块可以捕获这些异常。
如果启用了异常处理，代码却没有`try-catch`块，程序将崩溃并显示错误消息。

异常处理旨在检测很少发生的错误，并以优雅的方式从错误情形中恢复。
您可能认为只要不发生错误，异常处理就不会占用额外的时间，但不幸的是，事实并非总是如此。
程序可能需要进行大量（关于恢复信息的）簿记工作(bookkeeping)，才能知道如何在发生异常时进行恢复。
这种簿记花费的成本在很大程度上取决于不同的编译器。
一些编译器使用高效的基于表的方法，几乎没有额外开销，而其它编译器具有低效的基于代码的方法，或者依赖运行时类型识别（RTTI），会影响代码的其他部分。

进一步解释参照[ISO/IEC TR18015 Technical Report on C++ Performance](http://www.open-std.org/jtc1/sc22/wg21/docs/TR18015.pdf)。

下示例说明了为何需要簿记：

```cpp
// Example 7.48
class C1 {
public:
   ...
   ~C1();
};

void F1() {
   C1 x;
   ...
}

void F0() {
   try {
     F1();
   }
   catch (...) {
     ...
   }
}
```

函数`F1`本该在返回时调用对象`x`的析构函数。
但是如果在`F1`的某个地方发生异常怎么办？ 然后执行跳出了`F1`，函数没有返回。
由于`F1`被中断，因此无法完成清理。
现在，调用`x`的析构函数，就成了异常处理程序的责任。
要保证此操作可以执行，`F1`必需要保存所有相关的信息：
- 被调用的析构函数的所有信息
- 或可能需要的任何其他清理工作的信息

如果F1调用另一个函数，该函数又调用另一个函数等，并且如果在最里面的函数中发生异常，则异常处理程序需要有关函数调用链的所有信息，并且它需要通过函数调用向后跟踪路径，检查所有必要的清理工作。
这称为堆栈展开。

如果：
- `F1`调用另一个函数，该函数又调用另一个函数，等等，
- 并且如果在最里面的函数中发生异常

异常处理程序需要有关函数调用链的所有信息，并且它需要沿着函数调用的反向跟踪轨迹，检查所有必要的清理工作。
这称为堆栈展开。

所有函数都必须为异常处理程序保存一些信息，即使没有异常发生。
这就是为什么在某些编译器中异常处理可能代价高昂的原因。
如果你的应用程序异常处理不是必需的，你应该禁用它，这样代码更小，也更高效。
- 你可以通过关闭编译器中的异常处理选项，来禁用整个程序的异常处理。
- **你可以通过向函数原型添加`throw()`来禁用单个函数的异常处理：**
```cpp
void F1() throw();
```

**译者注:** `C++11`推荐使用 `noexcept`，`throw()`已经过时。


这允许编译器假定`F1`永远不会抛出任何异常，因此它不必保存函数`F1`的恢复信息。
然而，如果`F1`调用另一个可能抛出异常的函数`F2`，则`F1`必须检查`F2`抛出的异常，并在`F2`实际抛出异常的情况下调用`std::unexpected()`函数。
因此，只有当`F1`调用的所有函数都使用`throw()`限定符时，才能把`throw()`应用于`F1`。`throw()`对库函数很有用。

编译器区分**叶函数**和**帧函数**。
- 帧函数是调用至少一个其它函数的函数。
- 叶函数是没有调用任何其它函数的函数。

叶函数比帧函数简单，因为在某些情况下，叶函数可以省略掉堆栈展开信息：
- 如果可以排除异常
- 或者，如果在异常情况下没有任何真正需要清理的工作。

把被调用的所有函数内联，可以将帧函数转换为叶函数。
如果程序关键的最内层循环不包含对帧函数的调用，可以获得最佳性能。

尽管空`throw()`语句在某些情况下可以优化提高效率，但是没有必要添加诸如`throw(A,B,C)`之类的语句，来明确告诉函数可以抛出什么类型的异常。
实际上，如果添加这样的语句的话，编译器可能还需要添加额外的代码，来检查抛出的异常确实是指定的类型。
(参阅 Sutter：务实的异常规范, [Dr Dobbs Journal, 2002](http://drdobbs.com/architecture-and-design/184401544J)).

在某些情况下，即使在程序的最关键部分，使用异常处理也是最优策略。
通常出现这种情况的场景是：你希望程序能在出错时候恢复，且使用异常之外的其它错误恢复方法效率低下。
以下示例说明了这种情况：

```cpp
// Example 7.49
// Portability note: This example is specific to Microsoft compilers.
// It will look different in other compilers.
#include <excpt.h>
#include <float.h>
#include <math.h>
#define EXCEPTION_FLT_OVERFLOW 0xC0000091L

void MathLoop() {
   const int arraysize = 1000; unsigned int dummy;
   double a[arraysize], b[arraysize], c[arraysize];
   // Enable exception for floating point overflow:
   _controlfp_s(&dummy, 0, _EM_OVERFLOW);
   // _controlfp(0, _EM_OVERFLOW); // if above line doesn't work
   int i = 0; // Initialize loop counter outside both loops
   // The purpose of the while loop is to resume after exceptions:
   while (i < arraysize) {
     // Catch exceptions in this block:
     __try {
       // Main loop for calculations:
       for ( ; i < arraysize; i++) {
         // Overflow may occur in multiplication here:
         a[i] = log (b[i] * c[i]);
       }
     }
     // Catch floating point overflow but no other exceptions:
     __except (GetExceptionCode() == EXCEPTION_FLT_OVERFLOW ? EXCEPTION_EXECUTE_HANDLER : EXCEPTION_CONTINUE_SEARCH) {
       // Floating point overflow has occurred.
       // Reset floating point status:
       _fpreset();
       _controlfp_s(&dummy, 0, _EM_OVERFLOW);
       // _controlfp(0, _EM_OVERFLOW); // if above doesn't work
       // Re-do the calculation in a way that avoids overflow:
       a[i] = log(b[i]) + log(c[i]);
       // Increment loop counter and go back into the for-loop:
       i++;
     }
   }
}
```

假定`b[i]`和`c[i]`中的数字太大，以至于在乘法`b[i] * c[i]`中可能发生溢出，尽管这种情况很少发生。
上面的代码将在溢出的情况下捕获异常，并以一种效率更低，但却能避免溢出的方式重做计算。
即：取每个因子的对数，而不是乘积的对数，确保不会发生溢出，但计算时间会加倍。

这里，用来支撑异常处理所需的时间可以忽略不计，因为在关键的最内层循环中没有`try`块，也没有函数调用（除了`log`函数）。
`log`是一个库函数，我们假设它是非常优化的。
不管怎样，我们无法去除为`log`函数可能的异常处理提供的支撑（即记录额外的信息帮助在异常时恢复）。
当异常真的发生时，代价是高昂的。但这不是问题，因为我们知道这种情况很少发生。

在循环内判断溢出，并不需要任何代价，因为我们依靠的是微处理器硬件在溢出时引发的硬件异常。
如果程序中存在`try`块，操作系统会将异常处理转交到程序中的异常处理程序。

让我们看看这个例子中，有没有异常处理可能的替代方案。
我们可以检查`b[i]`和`c[i]`是否过大来判断溢出。
这将需要比较两个浮点数，因为它们必须在最内存的循环内，所以代价很高。
另一种可能的方法是，始终使用安全公式`a[i] = log(b[i])+ log(c[i]);`。
这会使`log`函数调用次数加倍，而且对数计算需要很长时间。
如果有一种方法可以在循环外检查异常，而不用检查所有数组元素，那么这可能是一个更好的解决方案。
- 如果所有因子都是从相同的几个参数生成的话，在循环之前进行这样的检查或许是可行的。
- 或者，如果计算结果由某个公式生成的单个结果，在循环后进行检查也许可行。

### 异常和向量代码

向量指令对于多个计算并行执行很有用。
这将在下面的第12章中描述。
异常处理与向量代码兼容性不好，因为向量中的单个元素可能导致异常，而其他向量元素可能正常运行。
你甚至可能会在未采用的分支中发生异常，仅仅是因为分支在向量代码中的特殊实现方式。
如果代码可以从向量指令中受益，那么最好禁用异常捕获，而依赖`NAN`和`INF`进行错误处理。
详见下文第7.34章。[www.agner.org/optimize/nan_propagation.pdf]() 文章中也有进一步讨论。

### 避免异常处理的开销

当不需要从错误情形中恢复时，异常处理时不必要的。
当错误发生时，如果你仅仅想要程序输出条错误消息后就终止运行，此时就没有理由使用 `try`, `catch`, 和 `throw`。这时，更有效的方法是：定义你自己的出错处理函数。该函数只需简单输出合适的错误消息，然后调用 `exit`。

当有分配的资源需要被清理是，调用 `exit` 也许并不安全，具体如后面所述。
还有其它不用异常的选项可供选择。
发现错误的函数可以返回一个错误代码，以供调用方来进行错误恢复，或者输出错误信息。

推荐使用系统化的，仔细斟酌过的方法来进行出错处理。
你**必须**要：
- 区分可恢复的错误和不可恢复的错误
- 确保分配的资源在出错时会被释放
- 输出适当的错误消息给用户

### 编写异常安全代码

假定一个函数以排斥模式打开了一个文件，然后一个错误情况发生导致程序在文件关闭前终止。程序终止后，文件会保持在锁定状态，直到计算机重启，用户会一直无法访问该文件。
位防止此类问题发生，你必须确保你的程序**异常安全**。
换言之，程序必须在异常或者出错情况下清理一切（资源）。可能需要清理的东西包括：

- 使用 new/malloc 分配的内存
- 窗口的句柄，图形刷等等的句柄
- 加了锁的互斥对象
- 打开的数据库连接
- 打开的文件和网络连接
- 需要被删除的临时文件
- 用户需要被保村的工作
- 其它的已分配的资源

C++ 处理清理工作的方式是使用析构函数。一个包含读写文件操作的函数可以被包装进入一个类。改类的析构函数要确保文件被关闭。
同样的方法也可以用在其它资源上，例如动态分配的内存，窗口，互斥体，数据库连接，等等。

C++ 异常处理系统会确保局部对象的所有析构函数被调用。
如果该带有析构函数的包装类负责处理所有的资源清理工作，程序是异常安全的。
但如果析构函数会引起别的异常，系统可能会失败。

如果你使用自己的出错处理系统，而不是使用异常处理，你不能确保所有的析构函数被调用，所有的资源被清理。
如果（你自己的）一个出错处理函数调用了`exit()`,`abort()`,`_endthread()`等，那么也不能保证所有的析构函数被调用。
不适用异常的情况下，处理不可恢复错误的安全方法是直接从函数返回。可能的话，该函数可以返回一个出错代码。或者把出错代码保存在一个全局对象里。调用方的函数必须检查该错误代码。
如果后一个函数也有一些东西要清理，那么它必须返回给它自己的调用者，依此类推。
